In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" #no annoyiing messages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd

In [2]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [3]:
# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Using Pandas and tf.data to make custom dataset

In [21]:
train_df = pd.read_csv("archive/train.csv")
test_df = pd.read_csv("archive/test.csv")
train_images = os.getcwd() + "/archive/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/archive/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [6]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [23]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

### Using Functional API

In [26]:
inputs = keras.Input(shape=(64,64,1))
x = layers.Conv2D( filters = 32, kernel_size = 3, 
                  padding = 'same', kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)

x = layers.Conv2D(64,3,kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)

x = layers.Conv2D(64,3,activation = 'relu', kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
x = layers.Conv2D(128,3, activation = 'relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)

x = layers.Dense(128,activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64,activation = 'relu')(x)

output1 = layers.Dense(10, activation='softmax',name='first_num')(x)
output2 = layers.Dense(10, activation='softmax', name='second_num')(x)

In [30]:
model = keras.Model(inputs = inputs, outputs = [output1,output2])

model.compile(
    optimizer = keras.optimizers.Adam(LEARNING_RATE),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [31]:
model.fit(train_dataset,epochs=5,verbose=2)
model.evaluate(test_dataset,verbose=2)

Epoch 1/5
1000/1000 - 26s - loss: 0.2264 - first_num_loss: 0.0903 - second_num_loss: 0.0866 - first_num_accuracy: 0.9721 - second_num_accuracy: 0.9722
Epoch 2/5
1000/1000 - 26s - loss: 0.1959 - first_num_loss: 0.0753 - second_num_loss: 0.0748 - first_num_accuracy: 0.9762 - second_num_accuracy: 0.9761
Epoch 3/5
1000/1000 - 27s - loss: 0.1698 - first_num_loss: 0.0647 - second_num_loss: 0.0630 - first_num_accuracy: 0.9792 - second_num_accuracy: 0.9798
Epoch 4/5
1000/1000 - 28s - loss: 0.1571 - first_num_loss: 0.0597 - second_num_loss: 0.0561 - first_num_accuracy: 0.9805 - second_num_accuracy: 0.9823
Epoch 5/5
1000/1000 - 28s - loss: 0.1419 - first_num_loss: 0.0523 - second_num_loss: 0.0502 - first_num_accuracy: 0.9830 - second_num_accuracy: 0.9842
313/313 - 3s - loss: 1.2490 - first_num_loss: 0.3867 - second_num_loss: 0.8244 - first_num_accuracy: 0.8991 - second_num_accuracy: 0.8011


[1.2489726543426514,
 0.38674262166023254,
 0.8244060277938843,
 0.8991000056266785,
 0.8010500073432922]